# Running the model

Setup a `feisty` integration.


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import yaml

import feisty

## Configure testcase to run

TODO: this is using the first column (which I believe is shallowest); want to add deepest column as well.
That will require refactoring the yaml file and also changing test_ds.

In [3]:
testcase = feisty.driver.simulate_testcase("tanh_shelf", "cyclic")

In [4]:
with open("matlab_first_timestep.yaml") as f:
    baselines = yaml.safe_load(f)


def compare(baselines_dict, da_or_ds, full_table=True, markdown_formatting=False):
    if markdown_formatting:
        if "dimname" in baselines_dict:
            print(f"| {baselines_dict['dimname']} | Matlab Value | Python Value | Rel Err |")
        else:
            print(f"|  | Matlab Value | Python Value | Rel Err |")
        print("| --- | --- | --- | --- |")
        seps = ["| ", " | ", " | ", " |"]
    else:
        seps = ["", ": ", ", ", ""]
    if "dimname" in baselines_dict:
        all_vars = da_or_ds[baselines_dict["dimname"]].data
    else:
        all_vars = baselines_dict["values"]
    for var in all_vars:
        if "dimname" in baselines_dict:
            da = da_or_ds
            sel_dict = {baselines_dict["dimname"]: var}
        else:
            da = da_or_ds[var]
            sel_dict = {}
        matlab_val = baselines_dict["values"][var]
        py_val = da.sel(sel_dict).data
        if matlab_val != 0:
            rel_err = (py_val - matlab_val) / matlab_val
            if rel_err < 0:
                rel_err = -rel_err
        else:
            if py_val == 0:
                rel_err = 0
            elif py_val > 0:
                rel_err = np.inf
            elif py_val < 0:
                rel_err = -np.inf
        if full_table or rel_err > 1e-2:
            print(
                f"{seps[0]}{var}{seps[1]}{matlab_val:10.4e}{seps[2]}"
                + f"{py_val:10.4e}{seps[2]}{rel_err:10.4e}{seps[-1]}"
            )

In [5]:
compare(
    baselines["day1"]["forcing"],
    testcase.forcing.isel(time=0, X=0, zooplankton=0),
    markdown_formatting=True,
)

|  | Matlab Value | Python Value | Rel Err |
| --- | --- | --- | --- |
| T_pelagic | 1.8200e+01 | 1.8200e+01 | 0.0000e+00 |
| T_bottom | 4.2000e+00 | 4.2000e+00 | 0.0000e+00 |
| zooC | 3.3287e+00 | 3.3287e+00 | 0.0000e+00 |
| poc_flux_bottom | 2.7847e-02 | 2.7847e-02 | 1.2459e-16 |
| zoo_mort | 7.7564e-01 | 7.7564e-01 | 0.0000e+00 |


## Run the model


In [6]:
%%time

nsteps = 1
make_plot = False
testcase.run(nsteps)
testcase.ds

CPU times: user 245 ms, sys: 4.17 ms, total: 249 ms
Wall time: 509 ms


<xarray.Dataset>
Dimensions:                    (X: 22, group: 9, time: 1, fish: 8, feeding_link: 19)
Coordinates:
  * X                          (X) float64 -0.5 -0.2381 0.02381 ... 4.738 5.0
  * group                      (group) <U12 'Sf' 'Sp' ... 'Ld' 'benthic_prey'
  * fish                       (fish) <U2 'Sf' 'Sp' 'Sd' 'Mf' ... 'Md' 'Lp' 'Ld'
  * feeding_link               (feeding_link) <U15 'Sf_Zoo' ... 'Ld_benthic_p...
    predator                   (feeding_link) <U2 'Sf' 'Sp' 'Sd' ... 'Ld' 'Ld'
    prey                       (feeding_link) <U12 'Zoo' ... 'benthic_prey'
Dimensions without coordinates: time
Data variables: (12/16)
    biomass                    (time, group, X) float64 9.999e-06 ... 0.002176
    T_habitat                  (time, fish, X) float64 18.2 18.2 ... 18.2 18.2
    ingestion_rate             (time, fish, X) float64 0.2152 ... 6.439e-07
    predation_flux             (time, fish, X) float64 5.737e-12 ... 0.0
    predation_rate             (time, fish, X) float64 5.737e-07 ... 0.0
    metabolism_rate            (time, fish, X) float64 0.04296 ... 0.00488
    ...                         ...
    recruitment_flux           (time, fish, X) float64 1.453e-09 ... 0.0
    fish_catch_rate            (time, fish, X) float64 0.0 0.0 ... 0.0008219
    encounter_rate_link        (time, feeding_link, X) float64 2.288 ... 0.0
    encounter_rate_total       (time, fish, X) float64 2.288 2.288 ... 6.439e-07
    consumption_rate_max_pred  (time, fish, X) float64 0.2375 0.2375 ... 0.01062
    consumption_rate_link      (time, feeding_link, X) float64 0.2152 ... 0.0

In [7]:
check_key = f"day{nsteps}"

if nsteps > 1:
    if make_plot:
        for group in test_ds.group.data:
            testcase.ds.biomass.sel(group=group).isel(X=0).plot()
        plt.ylim([1e-6, 1])
        plt.yscale("log")
        plt.legend(test_ds.group.data)

test_ds = testcase.ds.isel(X=0, time=-1)

In [8]:
compare(
    baselines[check_key]["T_habitat"],
    test_ds["T_habitat"],
    markdown_formatting=True,
)

| fish | Matlab Value | Python Value | Rel Err |
| --- | --- | --- | --- |
| Sf | 1.8200e+01 | 1.8200e+01 | 0.0000e+00 |
| Sp | 1.8200e+01 | 1.8200e+01 | 0.0000e+00 |
| Sd | 1.8200e+01 | 1.8200e+01 | 0.0000e+00 |
| Mf | 1.8200e+01 | 1.8200e+01 | 0.0000e+00 |
| Mp | 1.8200e+01 | 1.8200e+01 | 0.0000e+00 |
| Md | 4.2000e+00 | 4.2000e+00 | 0.0000e+00 |
| Lp | 1.8200e+01 | 1.8200e+01 | 0.0000e+00 |
| Ld | 4.2743e+00 | 4.2779e+00 | 8.4741e-04 |


In [9]:
compare(
    baselines[check_key]["metabolism"],
    test_ds["metabolism_rate"],
    markdown_formatting=True,
)

| fish | Matlab Value | Python Value | Rel Err |
| --- | --- | --- | --- |
| Sf | 4.2963e-02 | 4.2963e-02 | 1.6151e-16 |
| Sp | 4.2963e-02 | 4.2963e-02 | 1.6151e-16 |
| Sd | 4.2963e-02 | 4.2963e-02 | 1.6151e-16 |
| Mf | 1.4480e-02 | 1.4480e-02 | 1.1980e-16 |
| Mp | 1.4480e-02 | 1.4480e-02 | 1.1980e-16 |
| Md | 4.3744e-03 | 4.3744e-03 | 0.0000e+00 |
| Lp | 4.8804e-03 | 4.8804e-03 | 0.0000e+00 |
| Ld | 1.4837e-03 | 1.4842e-03 | 3.0973e-04 |


In [10]:
compare(
    baselines[check_key]["encounter"],
    test_ds["encounter_rate_link"],
    markdown_formatting=True,
)

| feeding_link | Matlab Value | Python Value | Rel Err |
| --- | --- | --- | --- |
| Sf_Zoo | 2.2885e+00 | 2.2885e+00 | 0.0000e+00 |
| Sp_Zoo | 2.2885e+00 | 2.2885e+00 | 0.0000e+00 |
| Sd_Zoo | 2.2885e+00 | 2.2885e+00 | 0.0000e+00 |
| Mf_Zoo | 2.9714e-01 | 2.9714e-01 | 1.8682e-16 |
| Mf_Sf | 1.9837e-06 | 1.9837e-06 | 0.0000e+00 |
| Mf_Sp | 1.9837e-06 | 1.9837e-06 | 0.0000e+00 |
| Mf_Sd | 1.9837e-06 | 1.9837e-06 | 0.0000e+00 |
| Mp_Zoo | 2.9714e-01 | 2.9714e-01 | 1.8682e-16 |
| Mp_Sf | 1.9837e-06 | 1.9837e-06 | 0.0000e+00 |
| Mp_Sp | 1.9837e-06 | 1.9837e-06 | 0.0000e+00 |
| Mp_Sd | 1.9837e-06 | 1.9837e-06 | 0.0000e+00 |
| Md_benthic_prey | 1.7232e-04 | 1.6423e-04 | 4.6954e-02 |
| Lp_Mf | 2.8618e-07 | 2.8618e-07 | 0.0000e+00 |
| Lp_Mp | 5.7237e-07 | 5.7237e-07 | 0.0000e+00 |
| Lp_Md | 0.0000e+00 | 0.0000e+00 | 0.0000e+00 |
| Ld_Mf | 8.9267e-08 | 8.9288e-08 | 2.2822e-04 |
| Ld_Mp | 1.7853e-07 | 1.7858e-07 | 2.2822e-04 |
| Ld_Md | 2.3805e-07 | 2.3810e-07 | 2.2822e-04 |
| Ld_benthic_prey | 

In [11]:
compare(
    baselines[check_key]["consumption"],
    test_ds["consumption_rate_link"],
    markdown_formatting=True,
)

| feeding_link | Matlab Value | Python Value | Rel Err |
| --- | --- | --- | --- |
| Sf_Zoo | 2.1520e-01 | 2.1520e-01 | 0.0000e+00 |
| Sp_Zoo | 2.1520e-01 | 2.1520e-01 | 0.0000e+00 |
| Sd_Zoo | 2.1520e-01 | 2.1520e-01 | 0.0000e+00 |
| Mf_Zoo | 4.2967e-02 | 4.2967e-02 | 0.0000e+00 |
| Mf_Sf | 2.8684e-07 | 2.8684e-07 | 1.8456e-16 |
| Mf_Sp | 2.8684e-07 | 2.8684e-07 | 1.8456e-16 |
| Mf_Sd | 2.8684e-07 | 2.8684e-07 | 1.8456e-16 |
| Mp_Zoo | 4.2967e-02 | 4.2967e-02 | 0.0000e+00 |
| Mp_Sf | 2.8684e-07 | 2.8684e-07 | 1.8456e-16 |
| Mp_Sp | 2.8684e-07 | 2.8684e-07 | 1.8456e-16 |
| Mp_Sd | 2.8684e-07 | 2.8684e-07 | 1.8456e-16 |
| Md_benthic_prey | 1.7091e-04 | 1.6294e-04 | 4.6586e-02 |
| Lp_Mf | 2.8616e-07 | 2.8616e-07 | 0.0000e+00 |
| Lp_Mp | 5.7232e-07 | 5.7232e-07 | 0.0000e+00 |
| Lp_Md | 0.0000e+00 | 0.0000e+00 | 0.0000e+00 |
| Ld_Mf | 8.8259e-08 | 8.8326e-08 | 7.5353e-04 |
| Ld_Mp | 1.7652e-07 | 1.7665e-07 | 7.5353e-04 |
| Ld_Md | 2.3536e-07 | 2.3554e-07 | 7.5353e-04 |
| Ld_benthic_prey | 

In [12]:
compare(
    baselines[check_key]["ingestion"],
    test_ds["ingestion_rate"],
    markdown_formatting=True,
)

| fish | Matlab Value | Python Value | Rel Err |
| --- | --- | --- | --- |
| Sf | 2.1520e-01 | 2.1520e-01 | 0.0000e+00 |
| Sp | 2.1520e-01 | 2.1520e-01 | 0.0000e+00 |
| Sd | 2.1520e-01 | 2.1520e-01 | 0.0000e+00 |
| Mf | 4.2968e-02 | 4.2968e-02 | 3.2298e-16 |
| Mp | 4.2968e-02 | 4.2968e-02 | 3.2298e-16 |
| Md | 1.7091e-04 | 1.6294e-04 | 4.6586e-02 |
| Lp | 8.5849e-07 | 8.5849e-07 | 0.0000e+00 |
| Ld | 4.9891e-05 | 4.7608e-05 | 4.5765e-02 |


In [13]:
compare(
    baselines[check_key]["predation"],
    test_ds["predation_rate"],
    markdown_formatting=True,
)

| fish | Matlab Value | Python Value | Rel Err |
| --- | --- | --- | --- |
| Sf | 5.7369e-07 | 5.7369e-07 | 1.8456e-16 |
| Sp | 5.7369e-07 | 5.7369e-07 | 1.8456e-16 |
| Sd | 5.7369e-07 | 5.7369e-07 | 1.8456e-16 |
| Mf | 3.7442e-07 | 3.7449e-07 | 1.7762e-04 |
| Mp | 7.4884e-07 | 7.4898e-07 | 1.7762e-04 |
| Md | 2.3536e-07 | 2.3554e-07 | 7.5353e-04 |
| Lp | 0.0000e+00 | 0.0000e+00 | 0.0000e+00 |
| Ld | 0.0000e+00 | 0.0000e+00 | 0.0000e+00 |


In [14]:
compare(
    baselines[check_key]["mortality"],
    test_ds["mortality_rate"],
    markdown_formatting=True,
)

| fish | Matlab Value | Python Value | Rel Err |
| --- | --- | --- | --- |
| Sf | 2.7397e-04 | 2.7397e-04 | 1.9787e-16 |
| Sp | 2.7397e-04 | 2.7397e-04 | 1.9787e-16 |
| Sd | 2.7397e-04 | 2.7397e-04 | 1.9787e-16 |
| Mf | 2.7397e-04 | 2.7397e-04 | 1.9787e-16 |
| Mp | 2.7397e-04 | 2.7397e-04 | 1.9787e-16 |
| Md | 2.7397e-04 | 2.7397e-04 | 1.9787e-16 |
| Lp | 2.7397e-04 | 2.7397e-04 | 1.9787e-16 |
| Ld | 2.7397e-04 | 2.7397e-04 | 1.9787e-16 |


In [15]:
compare(
    baselines[check_key]["fishing"],
    test_ds["fish_catch_rate"],
    markdown_formatting=True,
)

| fish | Matlab Value | Python Value | Rel Err |
| --- | --- | --- | --- |
| Sf | 0.0000e+00 | 0.0000e+00 | 0.0000e+00 |
| Sp | 0.0000e+00 | 0.0000e+00 | 0.0000e+00 |
| Sd | 0.0000e+00 | 0.0000e+00 | 0.0000e+00 |
| Mf | 8.2192e-04 | 8.2192e-04 | 1.3191e-16 |
| Mp | 8.2192e-05 | 8.2192e-05 | 0.0000e+00 |
| Md | 8.2192e-05 | 8.2192e-05 | 0.0000e+00 |
| Lp | 8.2192e-04 | 8.2192e-04 | 1.3191e-16 |
| Ld | 8.2192e-04 | 8.2192e-04 | 1.3191e-16 |


In [16]:
compare(
    baselines[check_key]["avail_energy"],
    test_ds["energy_avail_rate"],
    markdown_formatting=True,
)

| fish | Matlab Value | Python Value | Rel Err |
| --- | --- | --- | --- |
| Sf | 1.0767e-01 | 1.0767e-01 | 3.8666e-16 |
| Sp | 1.0767e-01 | 1.0767e-01 | 3.8666e-16 |
| Sd | 1.0767e-01 | 1.0767e-01 | 3.8666e-16 |
| Mf | 1.5598e-02 | 1.5598e-02 | 3.3365e-16 |
| Mp | 1.5598e-02 | 1.5598e-02 | 3.3365e-16 |
| Md | -4.2548e-03 | -4.2604e-03 | 1.3099e-03 |
| Lp | -4.8798e-03 | -4.8798e-03 | -0.0000e+00 |
| Ld | -1.4488e-03 | -1.4509e-03 | 1.4203e-03 |


In [17]:
compare(
    baselines[check_key]["growth"],
    test_ds["growth_rate"],
    markdown_formatting=True,
)

| fish | Matlab Value | Python Value | Rel Err |
| --- | --- | --- | --- |
| Sf | 1.0762e-01 | 1.0762e-01 | 1.2895e-16 |
| Sp | 1.0762e-01 | 1.0762e-01 | 1.2895e-16 |
| Sd | 1.0762e-01 | 1.0762e-01 | 1.2895e-16 |
| Mf | 0.0000e+00 | 0.0000e+00 | 0.0000e+00 |
| Mp | 1.5276e-02 | 1.5276e-02 | 8.6050e-09 |
| Md | 0.0000e+00 | 0.0000e+00 | 0.0000e+00 |
| Lp | 0.0000e+00 | 0.0000e+00 | 0.0000e+00 |
| Ld | 0.0000e+00 | 0.0000e+00 | 0.0000e+00 |


In [18]:
compare(
    baselines[check_key]["reproduction"],
    test_ds["reproduction_rate"],
    markdown_formatting=True,
)

| fish | Matlab Value | Python Value | Rel Err |
| --- | --- | --- | --- |
| Sf | 0.0000e+00 | 0.0000e+00 | 0.0000e+00 |
| Sp | 0.0000e+00 | 0.0000e+00 | 0.0000e+00 |
| Sd | 0.0000e+00 | 0.0000e+00 | 0.0000e+00 |
| Mf | 1.4534e-02 | 1.4534e-02 | 4.4798e-09 |
| Mp | 0.0000e+00 | 0.0000e+00 | 0.0000e+00 |
| Md | 0.0000e+00 | 0.0000e+00 | 0.0000e+00 |
| Lp | 0.0000e+00 | 0.0000e+00 | 0.0000e+00 |
| Ld | 0.0000e+00 | 0.0000e+00 | 0.0000e+00 |


In [19]:
compare(
    baselines[check_key]["recruitment"],
    test_ds["recruitment_flux"],
    markdown_formatting=True,
)

| fish | Matlab Value | Python Value | Rel Err |
| --- | --- | --- | --- |
| Sf | 1.4534e-09 | 1.4534e-09 | 4.4798e-09 |
| Sp | 0.0000e+00 | 0.0000e+00 | 0.0000e+00 |
| Sd | 0.0000e+00 | 0.0000e+00 | 0.0000e+00 |
| Mf | 1.0762e-06 | 1.0762e-06 | 0.0000e+00 |
| Mp | 1.0762e-06 | 1.0762e-06 | 0.0000e+00 |
| Md | 1.0762e-06 | 1.0762e-06 | 0.0000e+00 |
| Lp | 1.5276e-07 | 1.5276e-07 | 8.6050e-09 |
| Ld | 0.0000e+00 | 0.0000e+00 | 0.0000e+00 |


In [20]:
compare(
    baselines[check_key]["biomass"],
    test_ds["biomass"],
    markdown_formatting=True,
)

| group | Matlab Value | Python Value | Rel Err |
| --- | --- | --- | --- |
| Sf | 9.9993e-06 | 9.9993e-06 | 6.5108e-13 |
| Sp | 9.9978e-06 | 9.9978e-06 | 0.0000e+00 |
| Sd | 9.9978e-06 | 9.9978e-06 | 0.0000e+00 |
| Mf | 1.1076e-05 | 1.1076e-05 | 1.2618e-12 |
| Mp | 1.1076e-05 | 1.1076e-05 | 1.4114e-12 |
| Md | 1.1030e-05 | 1.1030e-05 | 5.0529e-06 |
| Lp | 1.0093e-05 | 1.0093e-05 | 1.3024e-10 |
| Ld | 9.9746e-06 | 9.9745e-06 | 2.0631e-06 |
| benthic_prey | 2.0985e-03 | 4.0885e-03 | 9.4828e-01 |
